For this part I have selected the following problems:
1. Four peaks problem
2. ContinuousPeaks problem
3. Flipflop problem

In [35]:
import mlrose_hiive
import numpy as np
import matplotlib.pyplot as plt
import time

Randomized Hill Climbing

In [ ]:
def randomized_hill_climbing(problem_fit):
    rhc = mlrose_hiive.random_hill_climb(problem=problem_fit, max_attempts=10)

Simulated Annealing

In [31]:
def simulated_annealing(problem_fit, search_space):


    fitness = mlrose_hiive.FitnessFunction(problem_fit)

    # Define the grid search object
    sa_grid_search = mlrose_hiive.GridSearch(
        algorithm=mlrose_hiive.SARunner,
        problem=problem_fit,
        experiment_name='sa_grid_search',
        iteration_list=[1000],
        seed=1234,
        grid=search_space,
        evaluation_metric=fitness,
        n_jobs=-1,
        verbose=False
    )

    # Run the grid search
    sa_grid_search.evaluate()
    print(sa_grid_search.best_parameters)
    sa = mlrose_hiive.SARunner(problem=problem_fit, experiment_name='sa_tsp',
                        iteration_list=[1000],
                        temperature_list=[1],
                        decay_list=[mlrose_hiive.GeomDecay],
                        seed=1234,
                        max_attempts=10
                        )

    # Run the simulated annealing algorithm
    sa_run_stats, sa_run_curves = sa.run()

    plt.plot(sa_run_curves['Iteration'], sa_run_curves['Fitness'])
    plt.xlabel('Iterations')
    plt.ylabel('Fitness')
    plt.title('Fitness vs. Iterations for TSP with GA')
    plt.show()

Genetic Algorithms

In [16]:
def genetic_algorithm(problem):
    ga = mlrose_hiive.GARunner(
        problem=problem,
        experiment_name='ga_experiment',
        seed=1234,
        iteration_list=[1000],
        population_sizes=[100],
        mutation_rates=[0.1],
        max_attempts=10,
        n_jobs=-1
    )

    # Run the genetic algorithm
    ga_run_stats, ga_run_curves = ga.run()

    # Print the best solution found
    # best_state, best_fitness = ga.best_state, ga.best_fitness
    # print('Best state:', best_state)
    # print('Best fitness:', best_fitness)
    return ga_run_stats, ga_run_curves

MIMIC

In [18]:
def mimic(problem):
    
    mimic = mlrose_hiive.MIMICRunner(
        problem=problem,
        experiment_name='mimic_experiment',
        seed=1234,
        iteration_list=[1000],
        population_sizes=[100],
        keep_percent_list=[0.2],
        max_attempts=10,
        n_jobs=-1
    )

    # Run the MIMIC algorithm
    mimic_run_stats, mimic_run_curves = mimic.run()

    # Print the best solution found
    # best_state, best_fitness = mimic.best_child()
    # print('Best state:', best_state)
    # print('Best fitness:', best_fitness)

In [91]:
def four_peaks():
    print("RUNNING FOUR PEAKS")
    max_attempts = 1000
    max_iterations = 75
    problem_size = 50

    fitness = mlrose_hiive.FourPeaks(t_pct=0.15)

    random_seed = 17

    sizes = [50, 75, 100, 125, 150, 175, 200, 225, 250, 275, 300]

    start = time.time()
    problem_fit = mlrose_hiive.DiscreteOpt(length=problem_size, fitness_fn=fitness, maximize=True)
    _, _, curve = mlrose_hiive.genetic_alg(problem=problem_fit, max_iters=max_iterations, pop_size=200, mutation_prob=0.40, random_state=random_seed, curve=True,max_attempts=max_attempts)
    print(f"Time for GA: {time.time()-start}")
    print(f"Function Evals for GA: {curve[-1,1]}")
    plt.plot(range(1,len(curve[:,0])+1), curve[:,0], label="GA")

    start = time.time()
    problem_fit = mlrose_hiive.DiscreteOpt(length=problem_size, fitness_fn=fitness, maximize=True)
    _, _, curve = mlrose_hiive.random_hill_climb(problem=problem_fit, max_iters=max_iterations, restarts=40, random_state=random_seed, curve=True, max_attempts=max_attempts)
    print(f"Time for RHC: {time.time()-start}")
    print(f"Function Evals for RHC: {curve[-1,1]}")
    plt.plot(range(1,len(curve[:,0])+1), curve[:,0], label="RHC")

    start = time.time()
    problem_fit = mlrose_hiive.DiscreteOpt(length=problem_size, fitness_fn=fitness, maximize=True)
    _, _, curve = mlrose_hiive.simulated_annealing(problem=problem_fit, max_iters=max_iterations, schedule=mlrose_hiive.algorithms.decay.GeomDecay(init_temp=200), random_state=random_seed, curve=True, max_attempts=max_attempts)
    print(f"Time for SA: {time.time()-start}")
    print(f"Function Evals for SA: {curve[-1,1]}")
    plt.plot(range(1,len(curve[:,0])+1), curve[:,0], label="SA")

    start = time.time()
    problem_fit = mlrose_hiive.DiscreteOpt(length=problem_size, fitness_fn=fitness, maximize=True)
    problem_fit.set_mimic_fast_mode(True)
    _, _, curve = mlrose_hiive.mimic(problem=problem_fit, max_iters=max_iterations, pop_size=450, keep_pct=0.33, random_state=random_seed, curve=True, max_attempts=max_attempts)
    print(f"Time for MIMIC: {time.time()-start}")
    print(f"Function Evals for MIMIC: {curve[-1,1]}")
    plt.plot(range(1,len(curve[:,0])+1), curve[:,0], label="MIMIC")

    plt.xlabel("Iterations")
    plt.ylabel("Fitness Score")
    plt.title(f"Fitness Score versus Iterations")
    plt.legend(loc="upper left")
    plt.savefig(f"plots/FourPeaks.png")
    plt.close()

    ga = []
    rhc = []
    sa = []
    mimic = []
    for i in sizes:
        fitness = mlrose_hiive.FourPeaks()
        problem_fit = mlrose_hiive.DiscreteOpt(length=i, fitness_fn=fitness, maximize=True)
        _, _, curve = mlrose_hiive.genetic_alg(problem=problem_fit, max_iters=2048, pop_size=200, mutation_prob=0.40, random_state=random_seed, curve=True)
        ga.append(curve[-1:,0])

        problem_fit = mlrose_hiive.DiscreteOpt(length=i, fitness_fn=fitness, maximize=True)
        _, _, curve = mlrose_hiive.random_hill_climb(problem=problem_fit, max_iters=512, restarts=40, random_state=random_seed, curve=True)
        rhc.append(curve[-1:,0])

        problem_fit = mlrose_hiive.DiscreteOpt(length=i, fitness_fn=fitness, maximize=True)
        _, _, curve = mlrose_hiive.simulated_annealing(problem=problem_fit, max_iters=8192, schedule=mlrose_hiive.algorithms.decay.GeomDecay(init_temp=100.0), random_state=random_seed, curve=True)
        sa.append(curve[-1:,0])

        problem_fit = mlrose_hiive.DiscreteOpt(length=i, fitness_fn=fitness, maximize=True)
        problem_fit.set_mimic_fast_mode(True)
        _, _, curve = mlrose_hiive.mimic(problem=problem_fit, max_iters=512, pop_size=450, keep_pct=0.33, random_state=random_seed, curve=True)
        mimic.append(curve[-1:,0])
    
    plt.plot(sizes, ga, label="GA")
    plt.plot(sizes, sa, label="SA")
    plt.plot(sizes, rhc, label="RHC")
    plt.plot(sizes, mimic, label="MIMIC")

    plt.xlabel("Problem Size")
    plt.ylabel("Fitness Score")
    plt.title(f"Fitness Score versus Problem Size")
    plt.legend(loc="upper left")
    plt.savefig(f"plots/FourPeaksProblemSize.png")
    plt.close()

In [92]:
four_peaks()

RUNNING FOUR PEAKS
Time for GA: 1.5670514106750488
Function Evals for GA: 15294.0
Time for RHC: 0.08740639686584473
Function Evals for RHC: 614.0
Time for SA: 0.0049915313720703125
Function Evals for SA: 150.0
Time for MIMIC: 4.920046329498291
Function Evals for MIMIC: 34285.0


In [48]:
def continuous_peaks():
    max_attempts = 3000
    max_iterations = 3000
    problem_size = 100 

    fitness = mlrose_hiive.ContinuousPeaks()
    problem_fit = mlrose_hiive.DiscreteOpt(length=problem_size, fitness_fn=fitness, maximize=True)

    start = time.time()
    problem_fit = mlrose_hiive.DiscreteOpt(length=problem_size, fitness_fn=fitness, maximize=True)
    _, _, curve = mlrose_hiive.genetic_alg(problem_fit, random_state=1, curve=True, max_iters=max_iterations, max_attempts=max_attempts, pop_size=150, mutation_prob=.01)
    print(f"Time for GA: {time.time()-start}")
    print(f"Function Evals for GA: {curve[-1,1]}")
    plt.plot(range(1,len(curve[:,0])+1), curve[:,0], label="GA")

    start = time.time()
    problem_fit = mlrose_hiive.DiscreteOpt(length=problem_size, fitness_fn=fitness, maximize=True)
    _, _, curve = mlrose_hiive.random_hill_climb(problem_fit, random_state=1, curve=True, max_iters=max_iterations, max_attempts=max_attempts)
    print(f"Time for RHC: {time.time()-start}")
    print(f"Function Evals for RHC: {curve[-1,1]}")
    plt.plot(range(1,len(curve[:,0])+1), curve[:,0], label="RHC")

    start = time.time()
    problem_fit = mlrose_hiive.DiscreteOpt(length=problem_size, fitness_fn=fitness, maximize=True)
    _, _, curve = mlrose_hiive.simulated_annealing(problem_fit, random_state=1, curve=True, max_iters=max_iterations, max_attempts=max_attempts, schedule=mlrose_hiive.GeomDecay(init_temp=10.0))
    print(f"Time for SA: {time.time()-start}")
    print(f"Function Evals for SA: {curve[-1,1]}")
    plt.plot(range(1,len(curve[:,0])+1), curve[:,0], label="SA")

    start = time.time()
    problem_fit = mlrose_hiive.DiscreteOpt(length=problem_size, fitness_fn=fitness, maximize=True)
    problem_fit.set_mimic_fast_mode(True)
    _, _, curve = mlrose_hiive.mimic(problem_fit, random_state=1, curve=True, max_iters=max_iterations, max_attempts=max_attempts, pop_size=100, keep_pct=.12)
    print(f"Time for MIMIC: {time.time()-start}")
    print(f"FUnction Evals for MIMIC: {curve[-1,1]}")
    plt.plot(range(1,len(curve[:,0])+1), curve[:,0], label="MIMIC")

    plt.xlabel("Iterations")
    plt.ylabel("Fitness Score")
    plt.title(f"Fitness Score versus Iterations")
    plt.legend(loc="upper left")
    plt.savefig(f"plots/ContinuousPeaks.png")
    plt.close()

    sizes = [50, 75, 100, 125, 150, 175, 200]
    ga = []
    rhc = []
    sa = []
    mimic = []
    for i in sizes:
        fitness = mlrose_hiive.ContinuousPeaks()
        problem_fit = mlrose_hiive.DiscreteOpt(length=i, fitness_fn=fitness, maximize=True)
        _, _, curve = mlrose_hiive.genetic_alg(problem_fit, random_state=1, curve=True, max_iters=max_iterations, max_attempts=max_attempts, pop_size=150, mutation_prob=.01)
        ga.append(curve[-1:,0])

        problem_fit = mlrose_hiive.DiscreteOpt(length=i, fitness_fn=fitness, maximize=True)
        _, _, curve = mlrose_hiive.random_hill_climb(problem_fit, random_state=1, curve=True, max_iters=max_iterations, max_attempts=max_attempts)
        rhc.append(curve[-1:,0])

        problem_fit = mlrose_hiive.DiscreteOpt(length=i, fitness_fn=fitness, maximize=True)
        _, _, curve = mlrose_hiive.simulated_annealing(problem_fit, random_state=1, curve=True, max_iters=max_iterations, max_attempts=max_attempts, schedule=mlrose_hiive.GeomDecay(init_temp=10.0))
        sa.append(curve[-1:,0])

        problem_fit = mlrose_hiive.DiscreteOpt(length=i, fitness_fn=fitness, maximize=True)
        problem_fit.set_mimic_fast_mode(True)
        _, _, curve = mlrose_hiive.mimic(problem_fit, random_state=1, curve=True, max_iters=max_iterations, max_attempts=max_attempts, pop_size=100, keep_pct=.12)
        mimic.append(curve[-1:,0])
    
    plt.plot(sizes, ga, label="GA")
    plt.plot(sizes, sa, label="SA")
    plt.plot(sizes, rhc, label="RHC")
    plt.plot(sizes, mimic, label="MIMIC")

    plt.xlabel("Problem Size")
    plt.ylabel("Fitness Score")
    plt.title(f"Fitness Score versus Problem Size")
    plt.legend(loc="upper left")
    plt.savefig(f"plots/ContinuousPeaksProblemSize.png")
    plt.close()

In [50]:
four_peaks()
continuous_peaks()

Time for GA: 32.10940432548523
Function Evals for GA: 201238.0
Time for RHC: 0.05199909210205078
Function Evals for RHC: 1011.0
Time for SA: 0.08598971366882324
Function Evals for SA: 1887.0
Time for MIMIC: 85.43018960952759
Function Evals for MIMIC: 101102.0
Time for GA: 56.4897096157074
Function Evals for GA: 453186.0
Time for RHC: 0.2079756259918213
Function Evals for RHC: 3045.0
Time for SA: 0.3111264705657959
Function Evals for SA: 4351.0
Time for MIMIC: 161.04733061790466
FUnction Evals for MIMIC: 303103.0
Time for GA: 0.5822474956512451
Function Evals for GA: 10264.0
Time for RHC: 0.0010104179382324219
Function Evals for RHC: 10333.0
Time for SA: 0.0010013580322265625
Function Evals for SA: 77.0
Time for MIMIC: 3.084150791168213
FUnction Evals for MIMIC: 15356.0


In [57]:
def flipflop():
    print("RUNNING FLIPFLOP")
    max_attempts = 1000
    max_iterations = 1000
    problem_size = 100

    fitness = mlrose_hiive.FlipFlop()

    start = time.time()
    problem_fit = mlrose_hiive.DiscreteOpt(length=problem_size, fitness_fn=fitness, maximize=True)
    _, _, curve = mlrose_hiive.genetic_alg(problem_fit, random_state=1, curve=True, max_iters=max_iterations, max_attempts=max_attempts, pop_size=200, mutation_prob=.1)
    print(f"Time for GA: {time.time()-start}")
    print(f"Function Evals for GA: {curve[-1,1]}")
    plt.plot(range(1,len(curve[:,0])+1), curve[:,0], label="GA")

    start = time.time()
    problem_fit = mlrose_hiive.DiscreteOpt(length=problem_size, fitness_fn=fitness, maximize=True)
    _, _, curve = mlrose_hiive.random_hill_climb(problem_fit, random_state=1, curve=True, max_iters=max_iterations, max_attempts=max_attempts)
    print(f"Time for RHC: {time.time()-start}")
    print(f"Function Evals for RHC: {curve[-1,1]}")
    plt.plot(range(1,len(curve[:,0])+1), curve[:,0], label="RHC")

    start = time.time()
    problem_fit = mlrose_hiive.DiscreteOpt(length=problem_size, fitness_fn=fitness, maximize=True)
    _, _, curve = mlrose_hiive.simulated_annealing(problem_fit, random_state=1, curve=True, max_iters=max_iterations, max_attempts=max_attempts)
    print(f"Time for SA: {time.time()-start}")
    print(f"Function Evals for SA: {curve[-1,1]}")
    plt.plot(range(1,len(curve[:,0])+1), curve[:,0], label="SA")

    start = time.time()
    problem_fit = mlrose_hiive.DiscreteOpt(length=problem_size, fitness_fn=fitness, maximize=True)
    problem_fit.set_mimic_fast_mode(True)
    _, _, curve = mlrose_hiive.mimic(problem_fit, random_state=1, curve=True, max_iters=max_iterations, max_attempts=max_attempts, pop_size=100, keep_pct=.1)
    print(f"Time for MIMIC: {time.time()-start}")
    print(f"Function Evals for MIMIC: {curve[-1,1]}")
    plt.plot(range(1,len(curve[:,0])+1), curve[:,0], label="MIMIC")

    plt.xlabel("Iterations")
    plt.ylabel("Fitness Score")
    plt.title(f"Fitness Score versus Iterations")
    plt.legend(loc="upper left")
    plt.savefig(f"plots/FlipFlop.png")
    plt.close()

    sizes = [50, 75, 100, 125, 150, 175, 200]
    ga = []
    rhc = []
    sa = []
    mimic = []
    for i in sizes:
        fitness = mlrose_hiive.FlipFlop()
        problem_fit = mlrose_hiive.DiscreteOpt(length=i, fitness_fn=fitness, maximize=True)
        _, _, curve = mlrose_hiive.genetic_alg(problem_fit, random_state=1, curve=True, max_iters=max_iterations, max_attempts=max_attempts)
        ga.append(curve[-1:,0])

        problem_fit = mlrose_hiive.DiscreteOpt(length=i, fitness_fn=fitness, maximize=True)
        _, _, curve = mlrose_hiive.random_hill_climb(problem_fit, random_state=1, curve=True, max_iters=max_iterations, max_attempts=max_attempts)
        rhc.append(curve[-1:,0])

        problem_fit = mlrose_hiive.DiscreteOpt(length=i, fitness_fn=fitness, maximize=True)
        _, _, curve = mlrose_hiive.simulated_annealing(problem_fit, random_state=1, curve=True, max_iters=max_iterations, max_attempts=max_attempts)
        sa.append(curve[-1:,0])

        problem_fit = mlrose_hiive.DiscreteOpt(length=i, fitness_fn=fitness, maximize=True)
        problem_fit.set_mimic_fast_mode(True)
        _, _, curve = mlrose_hiive.mimic(problem_fit, random_state=1, curve=True, max_iters=max_iterations, max_attempts=max_attempts, pop_size=100, keep_pct=.1)
        mimic.append(curve[-1:,0])
    
    plt.plot(sizes, ga, label="GA")
    plt.plot(sizes, sa, label="SA")
    plt.plot(sizes, rhc, label="RHC")
    plt.plot(sizes, mimic, label="MIMIC")

    plt.xlabel("Problem Size")
    plt.ylabel("Fitness Score")
    plt.title(f"Fitness Score versus Problem Size")
    plt.legend(loc="upper left")
    plt.savefig(f"plots/FlipFlopSize.png")
    plt.close()

In [58]:
flipflop()

RUNNING FLIPFLOP
Time for GA: 107.96222114562988
Function Evals for GA: 201217.0
Time for RHC: 0.6109828948974609
Function Evals for RHC: 1014.0
Time for SA: 0.80000901222229
Function Evals for SA: 1311.0
Time for MIMIC: 131.9964234828949
Function Evals for MIMIC: 101104.0
